##**Exploring MapReduce Options**
**Module 2, Section 3.2**  
**Block 9: Big Data Processing and NLP**

<a href="https://colab.research.google.com/github/datasciencepathways/hadoop_map_reduce/blob/main/tutorials/exploring_mapreduce_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
 
In our previous tutorial we went through the steps of running a simple MapReduce job on Google Colab. This tutorial will look at some of the parameters that allow you to control the exeuction of MapReduce jobs. 

The tutorial should not take more than 30 minutes to complete. 

The tutorial has been written in a way such that all commands work out of the box in Google Colab. However, if a particular command does not work or you get a weird error message, please add your question to the discussion forum.

The main sections of this tutorial are listed below. 


1. [Hadoop Install](#hadoop)
2. [Adding Functionality to Mapper Code](#code)
3. [Working with Real Datasets](#datasets) 
4. [Controlling Hadoop Job Parameters](#runtime)
5. [Analyzing the Output](#output)
6. [Measuring Performance](#performance)
7. [Conclusion](#end)


## <a name="hadoop"></a>Hadoop Install
Since the Google Colab environment is refreshed each time you open a new notebook, we will first need to install Hadoop in this VM instance. You can just follow the steps from the previous tutorial. For convenience, the sequence of commands for Hadoop installation is given below. 

In [ ]:
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.1/hadoop-3.3.1.tar.gz

In [ ]:
!tar -xzvf hadoop-3.3.1.tar.gz

In [ ]:
!ls hadoop-3.3.1/bin

In [4]:
!cp -r hadoop-3.3.1/ /usr/local/

In [ ]:
!readlink -f /usr/bin/java | sed 's/bin\/java//'

Now, use the folder navigation pane on the left to browse to the file `/usr/local/hadoop-3.3.0/etc/hadoop/hadoop-env.sh`. Double-click on the file to open it for editing. Uncomment the line begins with `export JAVA_HOME=` (should be line 54). Then add the Java path after the `=`

```bash
export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64/
```

In [ ]:
!/usr/local/hadoop-3.3.1/bin/hdfs namenode -format

In [ ]:
!/usr/local/hadoop-3.3.1/bin/hadoop

##<a name="code"></a>Adding Functionality to Mapper Code 

In this tutorial, we will again be working with the Word Count MapReduce program. 

Obtain the MapReduce code from the Block 9 Git repo. 

In [ ]:
!git clone https://github.com/datasciencepathways/hadoop_map_reduce.git

In this tutorial, we will utilize a slightly more advanced version of the mapper. The `hadoop_map_reduce/mapper_advanced.py` will still count all the words but skip some of the more common words like articles (a, an, the) and some common prepositions and verbs. This is often a useful strategy for dealing with large text datasets. Counting the common words can skew the final analysis and also make the program run for an unnecessarily longer time. You can take a look at the two versions of the mapper codes and see how we eliminate the common words from consideration. 

There is no change to the reducer code. 

As before, for convenience we will copy the mapper and reducer to the current directory. 

In [11]:
!cp hadoop_map_reduce/code/mapper_advanced.py .

In [12]:
!cp hadoop_map_reduce/code/reducer.py .

To be able to execute these codes, we will need to set the execute permission on the two files.

In [13]:
!chmod u+x mapper_advanced.py
!chmod u+rwx reducer.py

##<a name="datasets"></a>Working with Real Datasets

For any Hadoop job, we need to provide the names of an `input` and an `output` directory. The input directory is the place where the program is going to look for its input data. The output directory is the location of where the output is going to be written. 

These directories can be given any names. By convention, the names typically contain input/output as a suffix or a prefix. Let's create the input directory. 

In [14]:
!mkdir test_input

The output directory will be automatically created when the Hadoop job runs. 

In the previous tutorial, we created our own toy datasets to test our Hadoop program. In general though, you will be working with real datasets that are much larger. 

There are many free datasets that are available in the public domain. These are excellent candidates for evaluating your Hadoop MapReduce algorithms. Below are two useful sources

* [Kaggle Datasets](https://www.kaggle.com/datasets)  
* [Open-source Datasets for Text Classificatoin](https://analyticsindiamag.com/10-open-source-datasets-for-text-classification/)

Some of theses datasets are large and can take a long time to download. For your convenience we have copeid portions of these datasets into our course Git repository. When you cloned the repo, the datasets were downloaded with it. 

In this tutorial we will be working with the Lord of the Rings dataset which contains the full text of the [LOTR trilogy](https://en.wikipedia.org/wiki/The_Lord_of_the_Rings). 

Let's copy the files into our input directory. 

In [15]:
!cp hadoop_map_reduce/code/datasets/LOTR/*.txt test_input/

##<a name="runtime"></a>Controlling Hadoop Job Parameters 

Now we are all set to run our MapReduce Hadoop job. 

As a reminder, the general format for the command for running a MapReduce Hadoop job is as follows: 

```bash 
hadoop jar hadoop-streaming.jar \
-input name_of_input_file \
-output name_of_output_directory \
-file name_of_mapper_file \
-mapper the_mapper_cmd \
-file name_of_reducer_file \
-reducer the_reducer_cmd \
```

### Selecting input files
We will first run our Word Count MapReduce on the Fellowship of the Ring text. Note, we do not need to type the full name of the input file. We can use a wildcard to let the system figure out which files we want to use for this MapReduce job. When we say, `the_fellowship*.txt`, the system will look for all files in the input directory that begin with the word `the_fellowship`.  

In [ ]:
!rm -rf test_output
!/usr/local/hadoop-3.3.1/bin/hadoop jar /usr/local/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar -input test_input/the_fellowship*.txt -output test_output -file mapper_advanced.py  -file reducer.py  -mapper 'python mapper_advanced.py'  -reducer 'python reducer.py'

If everything went ok, you should see a whole bunch of output. The last line should give you the name of the output directory. 

Let's check the contents of the output directory. 

In [ ]:
!ls -ltr test_output

`part-00000` is the file that should contain the results of output of the word count program. You should see a recent timestamp on this file. Let's check the contents of that file. Because there are _many_ words in LOTR, we will just look at the count for the first 200 words using the Linux `head` command (`| head -200`)

In [ ]:
!cat test_output/part-00000 | head -200

##<a name="output"></a>Analyzing the Output 

Understandabaly, the output file is quite large and the data by itself does not give us any insight about the writing style of Tolkien. Typically, we would feed the word count data to another algorithm to analyze. Here, we will simply sort the output by occurrennce count to find out which words appear most frequently in The Fellowship of the Ring. 

We can use the Linux `sort` utility to sort the ouput. We will sort the words in descending order (`-r` flag) and just look at the top 100 (`| head -100`) 

In [ ]:
!sort -n -k 2 -r test_output/part-00000 | head -100

Does the output make sense? 

##<a name="performance"></a>Measuring Performance

Often times we are interested in how well the code performs. We can time the execution of the MapReduce code using the Linux `time` utility. To do this we will simply place the time command before the Hadoop MapReduce run command. This will re-run the MapReduce job and measure how long it took. 

In [ ]:
!rm -rf test_output
!time /usr/local/hadoop-3.3.1/bin/hadoop jar /usr/local/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar -input test_input/the_*.txt -output test_output -file mapper_advanced.py  -file reducer.py  -mapper 'python mapper_advanced.py'  -reducer 'python reducer.py'

The `time` commands gives us three times. The `real` time is total time elapsed and primarily what we are interested in. The `user` time, which is typically greater than `real` time is a measure of parallelism. The `sys` time is the time spent doing other work in the system. For more on the time command look at this [tutorial](https://www.journaldev.com/43534/time-command-in-linux)

## <a name="end"></a>Conclusion

That's it, you are done with this tutorial. 